In [27]:
import numpy as np
import pandas as pd
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
from surprise import BaselineOnly, KNNBaseline, KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from sklearn.metrics import log_loss
import os
from datetime import datetime

In [13]:
music = pd.read_csv('Digital_Music.csv', names=['user','item','rating'])

In [3]:
music_5folds = pd.read_json('Digital_Music_5.json', lines=True)

In [37]:
music_5folds.head()

,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
0,3426958910,NaN,5,"This is awesome to listen to, A must-have for ...","06 3, 2013",A2TYZ821XXK2YZ,Garrett,{'Format:': ' Audio CD'},Slayer Rules!,1370217600,True,3.0
1,3426958910,NaN,5,bien,"10 11, 2014",A3OFSREZADFUDY,Ad,{'Format:': ' Audio CD'},Five Stars,1412985600,True,NaN
2,3426958910,NaN,5,It was great to hear the old stuff again and I...,"02 11, 2014",A2VAMODP8M77NG,JTGabq,{'Format:': ' Audio CD'},SLAYER!!!!!!!!!!!!!!!!!!!!!,1392076800,True,NaN
3,3426958910,NaN,4,well best of's are a bit poison normally but t...,"12 7, 2013",AAKSLZ9IDTEH0,john F&#039;n doe,{'Format:': ' Audio CD'},slayer greatest hits! you mean everything righ...,1386374400,False,3.0
4,5557706259,NaN,5,What can I say? This is Casting Crowns!!!This ...,"06 12, 2016",A3OH43OZJLKI09,melinda a goodman,{'Format:': ' Audio CD'},"This is a good, blessing filled",1465689600,True,NaN


In [25]:
music_5folds['unixReviewTime'] > 

0          29745.0
1          70856.5
2          47416.0
3          42049.0
4         139094.5
5         120242.0
6         101987.5
7           2911.5
8           2728.0
9           2628.0
10          1779.0
11          1738.0
12          1737.0
13          1727.5
14        164871.5
15        147085.0
16        136452.0
17        124900.5
18        113829.0
19         87485.5
20         74790.5
21         71608.5
22         43080.0
23          1124.0
24          1124.0
25          1124.0
26        167257.0
27        165833.0
28        160832.5
29        159119.0
            ...   
169751    161560.0
169752    153179.0
169753    149540.0
169754    149097.5
169755    144035.0
169756    169446.5
169757    168967.5
169758    161623.0
169759    147885.0
169760    144933.5
169761    149264.0
169762    142454.0
169763    169749.0
169764    157860.5
169765    149264.0
169766    147524.0
169767    144809.0
169768    164568.5
169769    163189.5
169770    162413.0
169771    169549.5
169772    16

In [10]:
# It is to specify how to read the dataframe.
reader = Reader()

# create the traindata from the dataframe...
train_data = Dataset.load_from_df(music_5folds[['reviewerID','asin','overall']], reader)

# build the trainset from traindata to be compliant with surprise library's dataset format
trainset = train_data.build_full_trainset()


In [13]:
from surprise import SVD
algo = SVD()
algo.fit(trainset)

In [28]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


def get_ratings(predictions):
    actual = np.array([pred.r_ui for pred in predictions])
    pred = np.array([pred.est for pred in predictions])
    
    return actual, pred

def get_errors(predictions, print_them=False):

    actual, pred = get_ratings(predictions)
    error = log_loss(actual, pred, eps=1e-15, labels=[0,1])
    return error
    
def run_surprise(algo, trainset, verbose=True): 
    '''
        return train_dict, test_dict
    
        It returns two dictionaries, one for train and the other is for test
        Each of them have 3 key-value pairs, which specify ''rmse'', ''mape'', and ''predicted ratings''.
    '''
    start = datetime.now()
    # dictionaries that stores metrics for train and test..
    train = dict()
    #test = dict()
    
    # train the algorithm with the trainset
    st = datetime.now()
    print('Training the model...')
    algo.fit(trainset)
    print('Done. time taken : {} \n'.format(datetime.now()-st))
    
    # ---------------- Evaluating train data--------------------#
    st = datetime.now()
    print('Evaluating the model with train data..')
    # get the train predictions (list of prediction class inside Surprise)
    build_testset = trainset.build_testset()
    
    train_preds = algo.test(build_testset)
    
    print('Train preds are {}'.format(train_preds))
    # get predicted ratings from the train predictions..
    train_actual_ratings, train_pred_ratings = get_ratings(train_preds)
    
    # get top 10 item
    top_n = get_top_n(train_preds)

    
    # get 'log_loss from the train predictions.
    train_log_loss = get_errors(train_preds)
    print('time taken : {}'.format(datetime.now()-st))
    
    if verbose:
        print('-'*15)
        print('Train Data')
        print('-'*15)
        print("LogLoss : {}".format(train_log_loss))
    
    #store them in the train dictionary
    if verbose:
        print('adding train results in the dictionary..')
    train['log_loss'] = train_log_loss
    train['predictions'] = train_pred_ratings
    
    
    print('\n'+'-'*45)
    print('Total time taken to run this algorithm :', datetime.now() - start)
    
    # return two dictionaries train and test
    return train, top_n



In [15]:
music_5folds.shape

(169781, 12)

In [17]:
models_evaluation_train = dict()
models_evaluation_test = dict()

models_evaluation_train, models_evaluation_test

({}, {})

In [31]:
# initialize the model
#n_factors=100 # decrease the dimensionality to 100. q_i and p_u will be of 100 dimensions. This is a hyperparameter.
svd = SVD( biased=True, random_state=15, verbose=True)
svd_train_results,  svd_top_n= run_surprise(svd, trainset, verbose=True)

# Just store these error metrics in our models_evaluation datastructure
models_evaluation_train['svd'] = svd_train_results 
#models_evaluation_test['svd'] = svd_test_results

Training the model...
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Done. time taken : 0:00:10.835102 

Evaluating the model with train data..


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



time taken : 0:00:02.799550
---------------
Train Data
---------------
LogLoss : 1.2900430454411914e-17
adding train results in the dictionary..

---------------------------------------------
Total time taken to run this algorithm : 0:00:13.635108


In [30]:
# Print the recommended items for each user
for uid, user_ratings in svd_top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

A2TYZ821XXK2YZ ['B015U2LLIW', '3426958910', 'B01DQ6ON70', 'B01AUA1MC4', 'B00YIB0R9Q', 'B01EDA9EPE', 'B012BPYS7M', 'B01DQ6ONVG', 'B00MYPUDMY', 'B00YI4B79W']
A3OFSREZADFUDY ['B001I34TGY', 'B00A1G5VDO', 'B00RYGLWQ4', 'B001IX0E5Y', 'B00YIBA8J0', 'B004RCI9C8', 'B00EWS5IO4', 'B01AUA1MC4', '3426958910']
A2VAMODP8M77NG ['3426958910', 'B00THRTI8M', 'B00QKEMO5Y', 'B013CKV75G', 'B00YI4B79W']
AAKSLZ9IDTEH0 ['B001IX0E5Y', 'B009ZRZHWK', '3426958910', 'B00956UPSW', 'B015U2LLIW', 'B00CRCRNEA', 'B00TQTE8UE']
A3OH43OZJLKI09 ['B015HSUQRG', 'B00K5RU88W', '5557706259', 'B00192K3Y6', 'B00Q7SKT28']
A6JP9YYJCK3WO ['B004SBW17W', 'B009KUA7AE', 'B003KSBQFC', 'B00U370CJE', '5557706259', 'B004SBST2S', 'B0099G7KDQ', 'B004B5SFDY']
A20Z0JD766DPG6 ['B000WT62UQ', 'B000WT62UQ', 'B0048W3SQ4', 'B003KSBQFC', 'B0011W8V20', '5557706259', 'B0017TARAQ']
AXQ89O6YRT7AX ['B001G61C90', 'B00137Z0EK', 'B00137Z0EK', 'B001E5422C', '5557706259']
A1M379MK8MKGUF ['B0013D8BK4', 'B0013D8BK4', 'B001EUQHDO', 'B0019GAE3W', 'B001E5422C', '5557

A25B83TETXVD6M ['B00136J7ZE', 'B00136J7ZE', 'B000SF2UXW', 'B00137X8T4', 'B00137X8T4']
A33G5NB3THTY6G ['B001223OAI', 'B001YXWAL4', 'B004B3JHRO', 'B001NAZUM4', 'B0012ELM8W', 'B0012ELM8W', 'B00122KC8A', 'B000V6A53U', 'B000SF2UXW', 'B00137URZ2']
A3KLFLK1GQLGO8 ['B000SF2UXW', 'B00136RNQO', 'B00136RNQO', 'B00FAEQ22G', 'B00JGEU1AU']
A34XLJ6JF20K3A ['B000SF2UXW', 'B009D0Q5PM', 'B00137Y3AM', 'B001384GL2', 'B000TEGICQ', 'B00OTC1MB6', 'B000TEEJM2', 'B000V68U1Y', 'B00122KOBA', 'B00122KOBA']
A3DYLNS3B1ZUUO ['B000SFFCTG', 'B000W291YM', 'B000W291YM', 'B001NTI9X2', 'B002WQ0TN8', 'B0062IQPPI', 'B0099G71JY', 'B00NJT3GKK', 'B000VAISKI', 'B000W00IGY']
A1KL466UI2W31L ['B000VRSXS8', 'B000VRSXS8', 'B001YXWAB4', 'B000W176Y0', 'B00137QS2I', 'B001EAVJH8', 'B00ASETLUS', 'B0058U21N6', 'B00KZM0U3A', 'B009Y4FXT6']
A2STKYO3NMZL3P ['B000V61FXO', 'B00137RNZY', 'B000SFFCTG', 'B000YUHZBI', 'B000YUHZBI']
A19F4V98ZVOK7D ['B0011Z5IJG', 'B0011Z5IJG', 'B000SFFCTG', 'B001229952', 'B001GUJD2O']
A21X3U0ELWDPPQ ['B00123FMKM', 'B

A2E0RW04ZJVR0A ['B000TDB4R6', 'B000TD779K', 'B000TDUV0C', 'B000TDUV6G', 'B000W21ETC', 'B001232RGE', 'B00136PSAM', 'B00137QMV0', 'B000W21ETC', 'B001232RGE']
A2THNFFFYO1A6P ['B00330WKMA', 'B0058U21N6', 'B00GDL60JU', 'B01DOO46BM', 'B00940XGHG', 'B00GK8KIOY', 'B00OVKKUW8', 'B008GVTSXA', 'B0016CNY5Y', 'B0016CNY5Y']
A2ZGI9E6771F6I ['B00EH49FRE', 'B00EI6E17A', 'B00F2IZ9RY', 'B000TDB110', 'B000TDCZ0G']
A1GJR5BCAX97RW ['B000W23IE6', 'B000W23IE6', 'B000TDD1YU', 'B00137GB0M', 'B00137GB0M']
A3T2F40NCAB6TO ['B000TDD1YU', 'B000ULCSAY', 'B0070PVNRW', 'B01929H4VM', 'B0086HHEDA', 'B001229HGS', 'B0032PI4IA', 'B00136LKMC', 'B004NYLX8M', 'B001NTJU6M']
A2HPG8QACV8QAJ ['B000VZO4TW', 'B000VZO4TW', 'B00HZFDNLG', 'B000V8CBB2', 'B00137KQJO', 'B0161BPHKY', 'B000V64LIA', 'B001NSP4SQ', 'B000TDD1YU', 'B00FX8F6VM']
ACZ5MHW1M2NWI ['B000V66XJA', 'B005DOJ92S', 'B000TDD1YU', 'B0013F0E16', 'B0013F0E16']
A44XTWV3NXSY6 ['B000TDD1YU', 'B0011ZP3YG', 'B0011ZP3YG', 'B00136LQQ2', 'B00136LQQ2', 'B0030BDDWI', 'B000W0227C', 'B000W

A1TI2L994OYRZU ['B00BU2VHOA', 'B000VZV9Y0', 'B000VZV9Y0', 'B0012323YA', 'B0012323YA', 'B00122IR1O', 'B000W02Q1Y', 'B000W02Q1Y', 'B000TERLIQ', 'B0012287IW']
A2F0R1J0E2LHSJ ['B0011Z2Y7A', 'B0011Z2Y7A', 'B00HRN131I', 'B0011Z2Y5C', 'B0011Z2Y5C', 'B00137SRB8', 'B00137SRB8', 'B00HRN13DQ', 'B000W174H4', 'B00HRN141C']
A17814AD8EWTN4 ['B004EK6UGU', 'B002TYNSY0', 'B001232NP4', 'B001232NP4', 'B001J2WNE4', 'B004X0XR94', 'B004EKDE5A', 'B000TETN96', 'B001NDLQTW', 'B004BS87XE']
A33TRNCQK4IUO7 ['B00TV957BS', 'B01DUS7V78', 'B002PYCAKM', 'B01B8I3DHO', 'B001GH1ZQE', 'B000TETN96']
A200C7YQJ45LRR ['B00122N04I', 'B00122N04I', 'B000W176MM', 'B000W176MM', 'B00122KC4E', 'B00122KC4E', 'B00123BCKQ', 'B00123BCKQ', 'B001GH1ZQE', 'B000TETN96']
A35JR4D6FLXYRQ ['B000W176MM', 'B000W176MM', 'B00141YQW2', 'B00141YQW2', 'B005DL08L2', 'B008ZEIAYQ', 'B0096U8LH4', 'B00A3CX16K', 'B004BS87XE', 'B00110DT4C']
A3RV0VQKIM2HSW ['B000TETN96', 'B000W0TIRE', 'B000W0TIRE', 'B00122909W', 'B00122909W', 'B000V6ADNW']
A1TDPWNGX9OE6O ['B00

A3HMJ97JWEEG3S ['B000V63C7G', 'B000WM0OQQ', 'B001P75BT2', 'B004REYNG2', 'B000WLL8IA']
A1TIQNQJZ2LDNW ['B0011W3WRE', 'B0011W3WRE', 'B0011W3WRE', 'B0011W3WRE', 'B000V63C7G', 'B004BDCGPE', 'B001DQIXFO']
A1OHAF0U0SMHAR ['B00136LDQA', 'B00136LDQA', 'B0023RQSBI', 'B00136J966', 'B00136J966', 'B00136PLHW', 'B00136PLHW', 'B00137UTI2', 'B00137UTI2', 'B000V64Y5K']
A1HHW8MIX0C3NE ['B000V66URU', 'B002L4U5RG', 'B00330WF7K', 'B00122CB70', 'B00122CB70', 'B000VRSXJC', 'B000VRSXJC', 'B0011ZTCN4', 'B0011ZTCN4', 'B000V64Y5K']
A373ETRP1DHG9M ['B000V64UXQ', 'B000W126VS', 'B000W0YZBI', 'B00136NYNK', 'B00136NYNK', 'B000V63C7G']
A3OITRNUELR82W ['B00JLJ11QI', 'B000V63C7G', 'B00JLJ185C', 'B007JOM5XA', 'B008XCJ0KI']
A2UNDM1575MPH3 ['B000VBH2R2', 'B000VRPGVA', 'B000VRPGVA', 'B000V64Y5K', 'B002CNZVJS']
A2TYF5K1AW0O72 ['B000V64Y5K', 'B000V66URU', 'B000V66UV6', 'B000WH7EAU', 'B00136PU9G', 'B000WH7EAU', 'B00136PU9G', 'B001J2D906', 'B001LR6YFQ', 'B001NTZDGS']
A3JZJXR90732OD ['B000W23IE6', 'B000W23IE6', 'B000V64Y5K', 'B

...

APZ6UHI5TTON9 ['B00HZFDNLG', 'B00JHEHLYS', 'B00JGEU1AU', 'B004B3J6MA', 'B00JYKY9H2', 'B00FP1NNJY']
A3L8YUEWQM3K5K ['B005GXW1VW', 'B006FXDLWU', 'B007RE8F5O', 'B00DAKX0DG', 'B00F0AIBJC', 'B00GLP4BL2', 'B00GLP4DMO', 'B00GLP4AJK', 'B00C5ZMQG4', 'B00MR8YM4I']
A1T6Y0DE7WYQY ['B000W06VFQ', 'B000W04S8S', 'B00G7K9F3U', 'B00G7IVLLG', 'B00EL98FN0', 'B004B5SFDY']
A1N5JM9QQ3JFSS ['B0051QK8PU', 'B008E5YZ6S', 'B00AJRP4FU', 'B004B5SFDY', 'B00A20GWP0', 'B00AIGPIZI', 'B00AJMVTIG']
A1YHMU5RMI6DOL ['B004B51R0M', 'B00656QP2U', 'B006ZUES6G', 'B002LEV2SC', 'B006M4HVTQ']
A3IK65PO15GCG0 ['B000VZISJY', 'B000S55TTY', 'B0022F15MI', 'B004BC7MSQ', 'B0096RLJTY', 'B0022EZ3CC']
A2FSWI7Q6MZ9PX ['B004BC7MSQ', 'B0108UOUBG', 'B0108UP0EC', 'B0013L0HUI', 'B009G78G2C', 'B009G78L9U', 'B00EI6E17A', 'B00EUIMEDY', 'B00IDXXQMK', 'B00J9R8PJ8']
A3O920S2AIOKP3 ['B00F0AIBJC', 'B00CZF8B68', 'B004BC7MSQ', 'B00137GBC0', 'B001KPQSSC']
A1Z42SO1EE4L02 ['B00K5RU88W', 'B004BC7MSQ', 'B00GJ8H168', 'B00K5RU72O', 'B00GM5Y4YK', 'B01DELU46G', 'B00

A21FBKC8IQW3XB ['B00F0AIBJC', 'B00HNEQS3E', 'B00XTIFEVK', 'B00OD5FXPA', 'B00F0AIE4E', 'B006N9AW46', 'B00VMUSQF8', 'B009Y4FXT6', 'B00VMURTHE']
AEFUD8J2CKBAD ['B016E5Q134', 'B0013EWVEA', 'B00CZF8B68', 'B00EH49FRE', 'B0013F470A', 'B00AHXDGBA', 'B006OITIWS']
A3BCS8I6AZVHH3 ['B000X6NZHQ', 'B00O46VM8K', 'B00QWEESIS', 'B00X0BIF76', 'B014DIAMQG', 'B0161BPHKY', 'B01DOO46BM', 'B00I3499QQ', 'B00J9R8O4E', 'B00LBUNB36']
A2PKJ3VMEK4V9R ['B00GLP4DMO', 'B01BLYIT74', 'B007IXH79Y', 'B0092MKTL2', 'B006OITIWS']
A2JLPF1W67U99Y ['B00F2OIZLA', 'B0021QDXIW', 'B00PRUAV3O', 'B007B6VOII', 'B00EH49FRE', 'B006OITIWS']
A3VTK785JOOEJK ['B00AAAKHZ8', 'B00EYKUGUG', 'B00G2IAIVK', 'B0154NO8EG', 'B00SDOL5VI', 'B00NJMNLRK', 'B00MRQYZOM', 'B017JTOKV4', 'B006OITIWS', 'B00AAAKH80']
A2S6ZQ3Q8D1E42 ['B009EANFQ8', 'B006M4HWP4', 'B00JJOG5D4', 'B002AW98KO', 'B007TD92DC', 'B009G8GTFM', 'B006OITIWS']
A2RHDZ41XB4ZR0 ['B013AIHZLK', 'B006OISTVY', 'B00HUIHTKY', 'B01856MVR0', 'B00I37M1QI', 'B00OSCEOLW', 'B00HZ2PJMU']
AX6P637WVG6IA ['B01

A175NJU7HO0WUY ['B00AIN0UYA', 'B017EU18YA', 'B00QU1RZWO', 'B00KZPHG84', 'B019I4F1BQ', 'B00W8LYBBS', 'B00KZLNRD6', 'B00P7N0Y6K', 'B00JWFMVNS', 'B00NTW5SK8']
AYPU6P34IYXT ['B00AJ8ZCDI', 'B01C8Z8W7C', 'B00137YM62', 'B001EE4OF8', 'B00BWV9HUK', 'B00DVZK6AK', 'B00O0PLL0Y', 'B00137QNS2', 'B00LN2NNWQ']
A3IZOVD3W610TH ['B00AIVH4AK', 'B01HEIQ1JE', 'B00138543Q', 'B00AJ8ZCDI', 'B00HWSIONS']
A3G0ZHP2X2FUGC ['B00AJ8ZCDI', 'B00CZF8B68', 'B00SYTTHYY', 'B01BWA6YB0', 'B01B2H7TKI', 'B01BWA7K3G', 'B00U1T6J00', 'B01FTCOKF4', 'B01AXJH4EM']
A3FFHXQHQ3D3GG ['B00AJ8ZCDI', 'B00CBUJTTK', 'B00D052H7Q', 'B00CF6ZMG4', 'B00DYAS5PK', 'B00JWNYGLU']
A20Z7K2UJTA7PG ['B00G2PZSL8', 'B00NTW5SK8', 'B00GVZSTZQ', 'B00G7IVLLG', 'B00G722LDE', 'B00AJRP4FU']
A2Y478EJWBC7BK ['B014ELDZM0', 'B01FV9GS3M', 'B00YIB0R9Q', 'B00AN917QE', 'B000QZX5B0', 'B00FRCS90Y']
AFZZ8UJ534UF9 ['B01H5ZYNM8', 'B01929IHHC', 'B019I9ZVBQ', 'B00AN93Q0O', 'B00UC9EXKW', 'B0135P6PZA']
AOLXWTGF60RC ['B00UC9EXKW', 'B00AN93Q0O', 'B00V07DOUK', 'B01ENS16MK', 'B01CEZ

A3NR4BTTXMPBOP ['B00LAP3KS8', 'B00U0YCVCG', 'B00WB220EO', 'B00NB1IAJ8', 'B00IXBOP8G']
AKVEQ72PG52LT ['B00N23YW2Y', 'B00IXBOHWK', 'B00IXBOP8G', 'B00W48DUQ2', 'B00VIVB6T4', 'B00THRDSP6']
AAR5ICF2AGW3M ['B00W8DLF7Y', 'B00EPXY2F2', 'B00IXBOP8G', 'B006JAX0F2', 'B00120CWZI', 'B00137MIJU']
A91AOMO19WLGU ['B00NFDHUU2', 'B00IXBOHWK', 'B00IXZDP42', 'B000TH0M02', 'B00BGX1F4K']
A6KK29KYWHIT4 ['B00OSCEZCU', 'B00UCQ923S', 'B00YHTNBME', 'B01D0M04Y6', 'B00OIK3X9S', 'B00OSD7I5K', 'B00H3X5F8O', 'B00IXZ9WS0', 'B00P7DOZ4C', 'B00OSCEOLW']
AGYUDKFRU6LOL ['B00A70VSOK', 'B00IXZ9WS0', 'B00N5B61M2', 'B00K6B66LU', 'B00IONQJI2']
A28B0ZNBXP92BP ['B00TJ67YUK', 'B00LWC6P1S', 'B00IXZ9WS0', 'B00O8ZAX3C', 'B00TINQ9C8']
ASSFATPZCV08E ['B00NQLSJNA', 'B00QSCX42K', 'B01443NH5S', 'B00QHWTQ8C', 'B00HFEA0EK', 'B00TGVO4QA', 'B00QHWKAI2', 'B00POI9BTO', 'B00IXZ9QP4']
A2QW4YNZTHERR4 ['B01F9S6C54', 'B00C2XFZ6C', 'B0189YO1GC', 'B00QXY17RC', 'B00IXZ9QP4']
A255NFJWERWF7D ['B015TBO18G', 'B0158IALH0', 'B012CAS28M', 'B00IXARIPE', 'B0013

A2A7C92LC8HXD3 ['B00UV28VSA', 'B00W8DLF7Y', 'B010PE050U', 'B010PE0LRC', 'B010PDZS5S', 'B00WS6MQ8I', 'B001BKRX6W', 'B01AXJH4EM']
A7TQ4P8CTUABO ['B00137II8A', 'B001P7BL8C', 'B010W5W712', 'B006OZ0F7I', 'B00K2RD8NW']
A2G5MPHY34HGYH ['B007TG0K3A', 'B002QVGMOE', 'B0181SN52C', 'B0013AKNFI', 'B011DBWJ58', 'B00NERRC3Y', 'B00I3499QQ', 'B00137KHXY']
APUHN6MW2FEWW ['B019A712YG', 'B01DOO46BM', 'B015QOEPVE', 'B017OHGGRC', 'B011DBWJ58', 'B015LDMX14']
A3L2GWUADXZ1G2 ['B011DBVGH0', 'B017JTOHRQ', 'B00MMWUDHY', 'B016P0CA6U', 'B0161BPHKY']
A2JREJS24M35PT ['B000VWMTHE', 'B01CEZ5C88', 'B00137OFSW', 'B001BZD6GS', 'B001F3J598', 'B00F94W58W', 'B00136Q24I', 'B011DBV2NI', 'B002HFM1RQ', 'B004NYQDX2']
A1QNDKK0V140LU ['B00ET7ETK2', 'B01D1Z3ZLQ', 'B011ECYQC0', 'B0131ICFMS', 'B00M0Q34UA', 'B015KTSIP4']
AHQAG07IUO4ZO ['B00137VH60', 'B001581VRC', 'B00APCAVWK', 'B002EAP1QC', 'B000V7HJ2E', 'B004HRJ1ZW', 'B011N44KX4', 'B00123LEUY', 'B0070R90ZC', 'B001NS3TW4']
A3S1TAWQ4MQTMN ['B011N44KX4', 'B001223BPG', 'B001GO7930', 'B001

In [44]:
music_5folds.loc[music_5folds['asin'].isin(['B015U2LLIW', 'B01DQ6ON70', 'B01AUA1MC4', 'B00YIB0R9Q', 'B01EDA9EPE', 'B012BPYS7M', 'B01DQ6ONVG', 'B00MYPUDMY', 'B00YI4B79W'])]

,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
102906,B00MYPUDMY,NaN,5,This is my new favorite slipknot album. I have...,"10 21, 2014",A1HUN5L5NP6XI4,Bile,{'Format:': ' Audio CD'},Truly a GREAT album!,1413849600,False,NaN
102907,B00MYPUDMY,NaN,5,""".5: The Gray Chapter"" is great! This is anoth...","10 21, 2014",A3LZBOBV9H1HDV,Kris King,{'Format:': ' MP3 Music'},""".5: The Gray Chapter"" is great!",1413849600,False,2.0
102908,B00MYPUDMY,NaN,5,k,"08 9, 2017",A2IYYWBVZS40EN,Paul A.,{'Format:': ' Audio CD'},Five Stars,1502236800,True,NaN
102909,B00MYPUDMY,NaN,5,Fantastic album and a return to the sound that...,"08 1, 2017",A3V7PC7EDB8VA5,Metal Ecologist,{'Format:': ' MP3 Music'},Fantastic Return,1501545600,False,NaN
102910,B00MYPUDMY,NaN,5,"Great purchase, recommended seller!!!","05 3, 2017",A14IQQERCBD5TR,Alf V.,{'Format:': ' Audio CD'},Five Stars,1493769600,True,NaN
102911,B00MYPUDMY,NaN,5,I dig it. Better album than all hopeople is g...,"03 10, 2016",A1LXFDHH2V42BC,C. R. Smith,{'Format:': ' MP3 Music'},Better album than all hopeople is gone,1457568000,True,NaN
102912,B00MYPUDMY,NaN,5,great album they didn't disappoint,"03 9, 2016",A2KREV6WKLOQBE,Michael F.,{'Format:': ' Audio CD'},Five Stars,1457481600,True,NaN
102913,B00MYPUDMY,NaN,5,I've been a Slipknot fan since they first came...,"08 16, 2015",A1EJ9BMB058DRV,Shelley D. Bowen,{'Format:': ' Audio CD'},Up there with Slipknot's best,1439683200,False,NaN
102914,B00MYPUDMY,NaN,4,Still better than ever. One of my all time fav...,"07 31, 2015",A2TYZ821XXK2YZ,Garrett,{'Format:': ' Audio CD'},Maggot4Life,1438300800,True,NaN
102915,B00MYPUDMY,NaN,5,Slipknot has really managed to bring back the ...,"06 23, 2015",A3I9Q7TS5FQY0W,DJ,{'Format:': ' Audio CD'},WOW,1435017600,False,NaN


In [36]:
svd_top_n

defaultdict(list,
            {'A2TYZ821XXK2YZ': [('B015U2LLIW', 4.972232307353105),
              ('3426958910', 4.966286015379443),
              ('B01DQ6ON70', 4.916095750303851),
              ('B01AUA1MC4', 4.888422202704239),
              ('B00YIB0R9Q', 4.8614398543304995),
              ('B01EDA9EPE', 4.827985799891009),
              ('B012BPYS7M', 4.805591903622869),
              ('B01DQ6ONVG', 4.738507025883873),
              ('B00MYPUDMY', 4.61364082747535),
              ('B00YI4B79W', 4.473776288342683)],
             'A3OFSREZADFUDY': [('B001I34TGY', 5),
              ('B00A1G5VDO', 5),
              ('B00RYGLWQ4', 5),
              ('B001IX0E5Y', 4.962355859353897),
              ('B00YIBA8J0', 4.90277171374015),
              ('B004RCI9C8', 4.90271172516735),
              ('B00EWS5IO4', 4.840415645350813),
              ('B01AUA1MC4', 4.746919869236647),
              ('3426958910', 4.685816880083649)],
             'A2VAMODP8M77NG': [('3426958910', 5),
            

In [32]:
svd_train_results 

{'log_loss': 1.2900430454411914e-17,
 'predictions': array([4.96628602, 4.61364083, 4.47377629, ..., 4.95840418, 4.9105738 ,
        5.        ])}

In [35]:
cross_validate(algo,train_data,verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5513  0.5616  0.5741  0.5571  0.5615  0.5611  0.0075  
MAE (testset)     0.3301  0.3348  0.3398  0.3337  0.3370  0.3351  0.0032  
Fit time          8.30    7.70    9.12    10.58   12.08   9.56    1.59    
Test time         0.30    0.22    0.37    0.36    0.29    0.31    0.05    


{'test_rmse': array([0.55132685, 0.5615949 , 0.57412093, 0.55710121, 0.56150423]),
 'test_mae': array([0.3301338 , 0.33481336, 0.3398146 , 0.33370285, 0.33699919]),
 'fit_time': (8.304949045181274,
  7.702530145645142,
  9.122023105621338,
  10.579325199127197,
  12.081569910049438),
 'test_time': (0.2952451705932617,
  0.21967720985412598,
  0.3704242706298828,
  0.3571813106536865,
  0.2934861183166504)}